In [2]:
import pandas as pd
import numpy as np
import os
import math
import warnings
import gurobipy as gp
from gurobipy import GRB


warnings.filterwarnings('ignore')
pd.options.display.max_rows = 9999


- aisle interval= 50
- aisle length = 500   
   
- batch size = 25
- Number of batch = 24 

In [2]:
# file_list=os.listdir('./dataset/Orderlinelist')
# for f in file_list:
#     orderlinelist=pd.read_csv('./dataset/Orderlinelist/'+f,delimiter='\t',header=None,engine='python')
#     orderlinelist.columns=orderlinelist_column
#     if orderlinelist['LocationID'].max()>2881:
#         print(f)
    

In [3]:
path='./dataset'

dist_matrix=pd.read_csv(path+'/Distance matrix/DistanceMatrix_24SubAisles.txt',delimiter='\t',header=None,engine='python')
dist_matrix=dist_matrix.loc[:,:2880]

#orderlist
orderlist_clolumn=['OrderID', 'NumberOfOrderLines', 'ReleaseTime', 'DueTime' ,'FirstOrderLineID']

#orderline
orderlinelist_column=['OrderID', 'OrderLineID', 'AisleID' ,'CellID', 'LevelID', 'LocationID']


orderlist=pd.read_csv(path+'/Orderlist/OrderList_LargeProblems_1_21.txt',delimiter='\t',header=None,engine='python')
orderlist.columns=orderlist_clolumn


orderlinelist=pd.read_csv(path+'/Orderlinelist/OrderLineList_LargeProblems_1_21.txt',delimiter='\t',header=None,engine='python')
orderlinelist.columns=orderlinelist_column



In [4]:
print('distance matrix size:',dist_matrix.to_numpy().shape)
print('orderlist size:','(',len(orderlist),',',len(orderlist.columns),')')
print('orderlinelist size:','(',len(orderlinelist),',',len(orderlinelist.columns),')',end='\n\n')

print('@ orderlist info @')
display(orderlist.info())
print('@ orderlinelist info @')
display(orderlinelist.info())


distance matrix size: (2881, 2881)
orderlist size: ( 595 , 5 )
orderlinelist size: ( 2484 , 6 )

@ orderlist info @
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   OrderID             595 non-null    int64
 1   NumberOfOrderLines  595 non-null    int64
 2   ReleaseTime         595 non-null    int64
 3   DueTime             595 non-null    int64
 4   FirstOrderLineID    595 non-null    int64
dtypes: int64(5)
memory usage: 23.4 KB


None

@ orderlinelist info @
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   OrderID      2484 non-null   int64
 1   OrderLineID  2484 non-null   int64
 2   AisleID      2484 non-null   int64
 3   CellID       2484 non-null   int64
 4   LevelID      2484 non-null   int64
 5   LocationID   2484 non-null   int64
dtypes: int64(6)
memory usage: 116.6 KB


None

In [5]:
orderlinelist.head()

,OrderID,OrderLineID,AisleID,CellID,LevelID,LocationID,BatchID
0,1,1,13,7,0,1447,1
1,1,2,14,50,0,1610,1
2,1,3,8,6,0,846,1
3,1,4,1,15,0,15,1
4,1,5,5,31,0,511,1


In [6]:
orderlist.head()

,OrderID,NumberOfOrderLines,ReleaseTime,DueTime,FirstOrderLineID
0,1,7,0,3600,1
1,2,1,0,3600,8
2,3,1,0,3600,9
3,4,1,0,3600,10
4,5,2,0,3600,11


In [19]:
class OBP:
    def __init__(self,Orderlist,Orderlinelist,Batch_size):
        self.orderlist=Orderlist
        self.orderlinelist=Orderlinelist
        self.batch_size=Batch_size
        self.quotient=math.ceil(len(self.orderlist)/self.batch_size)

    def distance(self,aisle_last,max_aisle):
        return (2*50*aisle_last + 500*(max_aisle+1) + (max_aisle-1)*2*50 + 500*max_aisle)/2

    def routing_dist(self,batched_order):
 
        dist=0
        for i in range(self.quotient):
            aisle_last=batched_order[batched_order['BatchID']==i]['AisleID'].max()
            max_aisle=len(batched_order[batched_order['BatchID']==i]['AisleID'].unique())
            dist+=self.distance(aisle_last,max_aisle)
        return dist

    def random_batch(self):
        orderlist_ix=self.orderlist.index

        for i in range(self.quotient):
            if len(orderlist_ix) < self.batch_size:
                self.orderlinelist.fillna(i,inplace=True)
            else:
                orderlist_ix25=np.random.choice(orderlist_ix,self.batch_size)
                orderlist_ix=list(set(orderlist_ix)-set(orderlist_ix25))
            
                orderid_25=self.orderlist.loc[orderlist_ix25]['OrderID'].values
                self.orderlinelist.loc[self.orderlinelist['OrderID'].isin(orderid_25),'BatchID']=i
                
        return self.orderlinelist


    def FCFS(self):
        self.orderlinelist['BatchID']=None
        # batch_size=25

        for i in range(self.quotient):
            if len(self.orderlinelist[self.orderlinelist['BatchID'].isnull()]['OrderID'].unique()) < self.batch_size:
                self.orderlinelist.fillna(i,inplace=True)
            else:
                orderid_25=list(range(i*self.batch_size,(i+1)*self.batch_size))
                self.orderlinelist.loc[self.orderlinelist['OrderID'].isin(orderid_25),'BatchID']=i
        return self.orderlinelist


    
    def heu_function(self,orderlinelist_df):
        #TOP_AISLE
        
        x=orderlinelist_df['AisleID'].value_counts().index[0]
        orderlinelist_df['Y']=(orderlinelist_df['AisleID']-x)**2

        minimum_si=orderlinelist_df[['OrderID','Y']].groupby(by=['OrderID']).sum().min()[0]
        minimum_si_orders=orderlinelist_df[orderlinelist_df['Y']==minimum_si]['OrderID'].unique().tolist()

        return minimum_si_orders

    def heu_find_batchid(self,orderlinelist_df,minimum_si_orders):

        chosen_orders=minimum_si_orders
        orderlinelist=orderlinelist_df[~orderlinelist_df['OrderID'].isin(chosen_orders)]
        minimum_si_orders=self.heu_function(orderlinelist)
        chosen_orders=chosen_orders+minimum_si_orders

        if len(chosen_orders) >= self.batch_size:
            chosen_orders=chosen_orders[:self.batch_size]
        return chosen_orders

    def heuristic(self):

        orderline_df=self.orderlist.copy()
        orderlinelist_df=self.orderlinelist.copy()

        self.orderlinelist['BatchID']=None

        for i in range(self.quotient):
            chosen_orders=[]
            if len(orderlinelist_df['OrderID'].unique()) <self.batch_size:
                minimum_si_orders=orderlinelist_df['OrderID'].unique().tolist()
                self.orderlinelist.fillna(i,inplace=True)
            else:
                if len(chosen_orders) >= self.batch_size:
                    chosen_orders=chosen_orders[:self.batch_size]
                    
                    self.orderlinelist.loc[self.orderlinelist['OrderID'].isin(chosen_orders),'BatchID']=i
                    orderlinelist_df=orderlinelist_df[~orderlinelist_df['OrderID'].isin(chosen_orders)]
                    continue
                else:
                    while len(chosen_orders) < self.batch_size:
                        chosen_orders=self.heu_find_batchid(orderlinelist_df,chosen_orders)
                
                    self.orderlinelist.loc[self.orderlinelist['OrderID'].isin(chosen_orders),'BatchID']=i
                    orderlinelist_df=orderlinelist_df[~orderlinelist_df['OrderID'].isin(chosen_orders)]
                    
        return self.orderlinelist
    
    def get_total_distance(self,method):
        if method == 'FCFS':
            orderlinelist=self.FCFS()
        elif method == 'Heuristic':
            orderlinelist=self.heuristic()
        else: #random
            orderlinelist=self.random_batch()

        
        final_dist=self.routing_dist(orderlinelist)
        final_order=orderlinelist.drop_duplicates(subset=['OrderID','BatchID'])
        orderid=final_order.OrderID.tolist()
        batchid=final_order.BatchID.tolist()

        print('Method: ',method)
        print('total distance: ',final_dist)
        result={'Method':method,'OrderID':orderid,'BatchID':batchid,'Total_distance':final_dist}
        return result



In [22]:
obp=OBP(orderlist,orderlinelist,25)
methods=['random','FCFS','Heuristic']
total_dist=[]
for m in methods:
    result=obp.get_total_distance(m)
    total_dist.append(result['Total_distance'])
    display(result)
results=pd.DataFrame({'Method':methods,'Total_distance':total_dist})
results

Method:  random
total distance:  342700.0


{'Method': 'random',
 'OrderID': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154

Method:  FCFS
total distance:  347050.0


{'Method': 'FCFS',
 'OrderID': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,


Method:  Heuristic
total distance:  311500.0


{'Method': 'Heuristic',
 'OrderID': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  

,Method,Total_distance
0,random,342700.0
1,FCFS,347050.0
2,Heuristic,311500.0


# MIP

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Variation of mip1.py, with a focus on remote services
#
# When remote resources are tied to the optimization process, such as a token
# server, compute server, or Instant Cloud, extra care should be taken to
# ensure that such resources are released once they are no longer needed.
# Technically, such resources are managed by a gurobipy.Env object
# ("environment").  This example shows best practices for acquiring and
# releasing such shared resources via Env objects.
#
# See also https://www.gurobi.com/documentation/9.1/refman/environments.html


def populate_and_solve(m):

    #number of orders
    order_n=len(list(orderlist.index))
    #order list index
    i=range(0,order_n)
    #batch index=24, 
    batch_size=25
    #number of batches
    batch=math.ceil(order_n/25)
    #batch index
    k=range(0,batch)
    #aisle index
    m=sorted(list(orderlinelist.AisleID.unique()))
    #number of aisles
    s=len(m)
    #A_mi
    A=[[aisle_number],[order_index]]



    x=m.addVar(vtype=GRB.BINARY, name='x')
    y=m.addVar(vtype=GRB.BINARY, name='y')
    la=m.addVar(vtype=GRB.INTEGER, name='la')
    N=m.addVar(vtype=GRB.INTEGER, name='N')
    
    # Set objective
    aisle_length=500
    aisle_width=50
    
    m.setObjective((((la-1)*2*aisle_width + aisle_length*N + (aisle_length*2*aisle_width) + aisle_length*(N+1)/2)), GRB.MINIMIZE)
    
    # Add constraint: x + 2 y + 3 z <= 4
    const1 = m.addConstrs((x.sum(i,'*') <= batch_size for i in order_n), name='const1')
    const2 = m.addConstrs((x.sum('*',k) == 1 for k in batch), name='const2')
    const3 = m.addConstrs(y=max(A[][] for *x[][]), name='const3')
    const4 = m.addConstrs(, name='const4')
    const5 = m.addConstrs(la<=s, name='const5')
    const6 = m.addConstrs(, name='const6')
    const7 = m.addConstrs(, name='const7')
    
    # Optimize model
    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")


# Put any connection parameters for Gurobi Compute Server, Gurobi Cluster
# Manager or Gurobi Token server here, unless they are set already
# through the license file.

connection_params = {
    # For Compute Server you need at least this
    #       "ComputeServer": "<server name>",
    #       "UserName": "<user name>",
    #       "ServerPassword": "<password>",
    # For Cluster Manager you need at least this
    #       "CSManager": "<manager name>",
    #       "CSAPIAccessID": "<access ID>",
    #       "CSAPISecret": "<secret>",
    # For Instant cloud you need at least this
    #       "CloudAccessID": "<access id>",
    #       "CloudSecretKey": "<secret>",
}

with gp.Env(params=connection_params) as env:
    # 'env' is now set up according to the connection parameters.
    # The environment is disposed of automatically through the context manager
    # upon leaving this block.
    with gp.Model(env=env) as model:
        # 'model' is now an instance tied to the enclosing Env object 'env'.
        # The model is disposed of automatically through the context manager
        # upon leaving this block.
        try:
            populate_and_solve(model)
        except:
            # Add appropriate error handling here.
            raise